In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import  train_test_split
import matplotlib.pyplot as plt

In [2]:
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    data = np.array(df.iloc[:100, [0, 1, -1]])
    for i in range(len(data)):
        if data[i,-1] == 0:
            data[i,-1] = -1
    # print(data)
    return data[:,:2], data[:,-1]

In [18]:
X, y = create_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=13)

In [19]:
A=[[1  ,10], [100,1000]]
B=[[1,2], [3,4]]

In [20]:
np.dot(A,B)

array([[  31,   42],
       [3100, 4200]])

In [21]:
A=[[100,1000]]
B=[[1,2], [3,4]]

In [22]:
class SVC():
    def __init__(self,C=1.0,max_iter=1000,kernel='linear',epsilon=0.001,c=0.5,d=3):
        self.C=C
        self.kernel=kernel
        self.epsilon=epsilon
        self.max_iter=max_iter
        self.c=c
        self.d=d
        if self.C<0:
            raise ValueError('penalty term must be positive ,bot got (C=%r)' % self.C)
        if self.kernel not in ('linear','polynomial'):
            raise ValueError('we only support linear kernal or polynomial kernal fun now')
    
    
    # train  a svm model
    def fit(self,x,y):
        self.x=x
        self.Y=y
        self.b=0.0
        self.m=len(x)
        self.n=len(x[0])
        self.alpha=[0.0]* self.m
        self.E=np.array([self._E_(i) for i in range (self.m)])
        self._smo()
        
    #check if the alpha[i] satify the kkt condition
    def _safisfy_KKT_(self,i):
        tmp=self.Y[i]*self._g_(i)
        if abs(self.alpha[i])<self.epsilon:
            return tmp>=1
        elif abs(self.alpha[i]-self.C) < self.epsilon:
            return tmp<=1
        else:
            return abs(tmp-1)< self.epsilon
    
    ## the predict val of sample[i]
    def _g_(self,i):
        k=np.array([self._k_(i,j) for j in range(self.m) ])
        return np.dot(self.alpha * self.Y,k)+self.b
    
    def _k_(self,i,j):
        if isinstance(i,int):
            if self.kernel=='linear':
                return np.inner(self.x[i],self.x[j])
            elif self.kernel=='polynomial':
                return (np.dot(self.x[i],slef.x[j])+self.c)**self.d
        else: # i is a vector
            if self.kernel == 'linear':
                return np.inner(i,self.x[j])
            elif self.kernel=='polynomial':
                return (np.dot(i,self.x[j])+self.c)**self.d
    
    ###compute predict val-true val for sample_i
    def _E_(self,i):
        return self._g_(i)-self.Y[i]
    
    ### select 2 alpha to update:
    # if return none ,stop
    def _select_2(self):
        #first alpha:find not satisfy kkt condition from all 0<alpha[i]<c
        alpha_1_index=[i for i in range(self.m) if self.alpha[i] >
                      self.epsilon and abs(self.alpha[i]-self.C) > self.epsilon]
        alpha_2_index=list(set(list(range(self.m)))-set(alpha_1_index))
        
        # first check 0<alpha[i]<c ,if they satisfy kkt all,then check other alpha
        alpha_1_index.extend(alpha_2_index ) 
        
        for i in alpha_1_index:
            if self._safisfy_KKT_(i):
                continue
            imax=(0,0) #store |E1-E2| ,index of alpha2
            E1=self.E[i]
            alpha_1_index.remove(i)
            for j in alpha_1_index:
                E2 = self.E[j]
                if abs(E1-E2)> imax[0]:
                    imax=(abs(E1-E2),j)
            return i,imax[1]
    
    #classical squecntial minimal optimization algorithm
    def _smo(self):
        for _ in range(self.max_iter):
            if (_+1)%100==0 :
                print('iter round :%d' % (_+1))
            
            tup = self._select_2()
            if tup is None :
                print('satisfy stop condition')
                break
            else :
                i1,i2=tup
            
            E1,E2=self.E[i1],self.E[i2]
            eta=self._k_(i1,i1)+self._k_(i2,i2)-2*self._k_(i1,i2)
#             print('-------')
#             print(self.alpha[i2])
#             print('-------')
            alpha2_new_unc=self.alpha[i2]+self.Y[i2]*(E1-E2)/eta
#             print('-------')
#             print(alpha2_new_unc)
#             print('-------')           
            if self.Y[i1]==self.Y[i2]:
                L=max(0,self.alpha[i2]+self.alpha[i1]-self.C)
                H=min(self.C,self.alpha[i2]+self.alpha[i1])
            else:
                L=max(0,self.alpha[i2]-self.alpha[i1])
                H=min(self.C,self.C+self.alpha[i2]-self.alpha[i1])
#             alpha2_new = H if alpha2_new_unc > H else L if alpha2_new_unc < L else alpha2_new_unc
#             print(H)
#             print(L)
#             print(alpha2_new_unc)
            alpha2_new= H if alpha2_new_unc >H  else L if alpha2_new_unc<L else alpha2_new_unc
            alpha1_new=self.alpha[i1]+self.Y[i1]*self.Y[i2]*(self.alpha[i2]-alpha2_new)
            
            #update b
            b1_new=-E1-self.Y[i1]*self._k_(i1,i1)*(alpha1_new-self.alpha[i1])\
            -self.Y[i2]*self._k_(i2,i1)*(alpha2_new - self.alpha[i2])+self.b
            b2_new=-E2-self.Y[i1]*self._k_(i1,i2)*(alpha1_new-self.alpha[i1])\
            -self.Y[i2]*self._k_(i2,i2)*(alpha2_new - self.alpha[i2])+self.b
            
            # update b ,alpha1,alpha2,E1,E2
            if alpha1_new >0 and alpha1_new<self.C:
                self.b = b1_new
            elif alpha2_new >0 and alpha2_new<self.C:
                self.b=b2_new
            else:
                self.b=(b1_new+b2_new)/2
            
            self.alpha[i1]=alpha1_new
            self.alpha[i2]=alpha2_new
            self.E[i1]=self._E_(i1)
            self.E[i2]=self._E_(i2)
            
    def _predict_(self,x):
        res=self.b
        for i in range(self.m):
            res+=self.alpha[i]*self.Y[i]*self._k_(x,i)
        return 1 if res >0 else -1

    def predict(self,x):
        res=[self._predict_(x1) for x1 in x]
        return res
            

In [23]:
svc = SVC()
X = np.array([
    [3,3],
    [4,3],
    [1,1]])
Y = np.array([1, 1, -1])
svc.fit(X, Y)
print(svc.predict([
    [1, 1/2],
    [3, 5],
    [6, 2],
    [3,4]
]))

satisfy stop condition
[-1, 1, 1, 1]


In [24]:
def pred_score(y_pred,y_true):
    count=0
    for i in range(len(y_pred)):
        if y_pred[i]==y_true[i]:
            count+=1
    return count/len(y_pred)
    

In [25]:
svc.fit(X_train,y_train)
pred_score(svc.predict(X_test),y_test)


D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:98: RuntimeWarning: divide by zero encountered in double_scalars


iter round :100
iter round :200
iter round :300
iter round :400
iter round :500
iter round :600
iter round :700
iter round :800
iter round :900
iter round :1000


0.76

In [27]:
smo=SVC(max_iter=5)
smo.fit(X_train,y_train)
pred_score(smo.predict(X_test),y_test)

0.96